In [1]:
"""

MONTHLY DATA NOW

"""

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import pandas as pd
import yfinance as yf
import fredapi as fa
from datetime import date, timedelta
from Modules.Exposure_Report import Exposure
import numpy as np
import matplotlib.pyplot as plt
import Modules.Useful_Functions as u
from statsmodels.tsa.stattools import adfuller
import Modules.DataHub as da

# sectors_and_rates
cutoff = .7

#factor_df = pd.read_csv("2006.csv", index_col="Date")

# .drop(columns = ["MOMENTUM"])
#factor_df = pd.read_csv("regular_factors_and_rates.csv", index_col = "Date").drop(columns = ["MOMENTUM"])
factor_df = da.get_Dataset('factor', ['real_yield', 'yield_curve'])

train_df = factor_df[:int(len(factor_df)*cutoff)]
test_df = factor_df[int(len(factor_df)*cutoff):]

forward = 30

exposure = Exposure(train_df, 252, forward, ['real_yield', 'yield_curve'])
for_scores = Exposure(factor_df, 252, forward, ['real_yield', 'yield_curve'])

ry_scores_test_monthly = for_scores.monthly_scores['real_yield']
yc_scores_test_monthly = for_scores.monthly_scores['yield_curve']
ry_scores_test_monthly = ry_scores_test_monthly.loc[ ry_scores_test_monthly.index > train_df.index[-1]]
yc_scores_test_monthly = yc_scores_test_monthly.loc[ yc_scores_test_monthly.index > train_df.index[-1]]

train_results_df = exposure.forward_returns_monthly.dropna().copy()
update_df = exposure.forward_returns_monthly.dropna().copy()


#update_df

#u.current_rates('m')


In [2]:


update_df


'/Users/rhys/Desktop/740/Reference_Data/macro_data.csv'

In [2]:

def closest_combos(combo, in_index = False):

    assert isinstance(combo, tuple)
    
    min_error = 100
    errors = {}

    for rates in train_results_df.index:
        
        ry_error = abs(combo[0] - rates[0])
        yc_error = abs(combo[1] - rates[1])
        error = ry_error + yc_error
        
        
        errors[rates] = error
        
        if error <= min_error and error > 0:
            min_error = error
    
    mins = list(filter(lambda x: errors[x]==min_error, errors))
    
    if in_index:
        mins.append(combo)
    
    return mins


ok = closest_combos((1,1), True)

In [12]:

predicted_best_sector = []
exp_ret = []
scores = []
top_n_sectors = 1

for num, (ry, yc) in enumerate(zip(ry_scores_test_monthly, ry_scores_test_monthly)):
    
    if (ry, yc) in train_results_df.index:
        combos = closest_combos((ry, yc), True)
    else:
        combos = closest_combos((ry, yc))
        
    top_secs = train_results_df.loc[combos].mean().sort_values()[-top_n_sectors:]
        
    predicted_best_sector.append(top_secs.index.to_list())
    exp_ret.append(top_secs.mean())
    scores.append((ry,yc))
    
    

In [13]:


big_df = pd.DataFrame(index = ry_scores_test_monthly.index)
big_df['scores'] = scores
big_df['exp_rets_predicted'] = exp_ret
big_df['pred_top_secs'] = predicted_best_sector



In [14]:


actual_results_exposure = Rate_Exposures(test_df, 252, forward, ['Real Yield', 'Yield Curve'])
actual_returns = actual_results_exposure.forward_returns_monthly.iloc[:,:-2]
actual_returns = actual_returns.loc[ (actual_returns.index >= big_df.index[0]) & (actual_returns.index <= big_df.index[-1])]



In [15]:


actual_secs = []
actual_ret = []
predicted_sector_performance = []
correct_sector_performance = []

for d in actual_returns.index:

    top_actual_secs = actual_returns.loc[str(d)].sort_values()
    actual_secs.append(top_actual_secs.index[-top_n_sectors:].to_list())
    
    actual_ret.append(top_actual_secs[-top_n_sectors:].mean())
    predicted_sector_performance.append(actual_returns.loc[str(d)][list(big_df['pred_top_secs'].loc[str(d)])].mean())

actual_secs = pd.Series(actual_secs, index = actual_returns.index)
actual_ret = pd.Series(actual_ret, index = actual_returns.index)

big_df['actual_top_secs'] = actual_secs
big_df['actual_rets'] = actual_ret

big_df = big_df.dropna()


In [16]:


correct_sectors = []
mean_squared_error_return = []
return_error_regular = []


for pred, act in zip(big_df['pred_top_secs'], big_df['actual_top_secs']):
    sect = set(pred).intersection(set(act))
    correct_sectors.append(len(sect))
    

for pred, act in zip(big_df['exp_rets_predicted'], big_df['actual_rets']):
    
    mean_squared_error_return.append((pred-act)**2)
    return_error_regular.append(pred-act) 
    
big_df['correctly_chosen_sectors'] = correct_sectors
#big_df['correctly_chosen_sectors_above_market'] = correct_sectors_above_market
big_df['predicted_sector_performance'] = predicted_sector_performance
#big_df['correct_sector_performance'] = actual_results['actual_rets']
big_df['mean_squared_error_return_prediction'] = mean_squared_error_return
big_df['return_error_regular'] = return_error_regular

big_df = big_df.dropna()

In [19]:


#big_df.predicted_sector_performance.loc[big_df.correctly_chosen_sectors >= 0].hist()

#big_df.predicted_sector_performance.hist()
#big_df.correctly_chosen_sectors.plot()


big_df.predicted_sector_performance.sort_values()




2020-03   -0.585818
2020-04   -0.560830
2020-07   -0.294533
2019-11   -0.236394
2018-10   -0.150986
2019-10   -0.115462
2020-11   -0.080216
2020-08   -0.048852
2019-12   -0.013976
2019-08   -0.012933
2019-09    0.001410
2019-02    0.013911
2020-06    0.015043
2021-02    0.015100
2019-07    0.020637
2019-06    0.040310
2018-12    0.050063
2019-01    0.072876
2019-04    0.131160
2018-11    0.134363
2019-05    0.184186
2018-08    0.193350
2018-09    0.208379
2019-03    0.283205
2020-02    0.287279
2020-09    0.375990
2020-10    0.382405
2020-12    0.396638
2020-01    0.428848
2020-05    0.435553
2021-01    0.462047
Freq: M, Name: predicted_sector_performance, dtype: float64